### Calculate the luminosity during each segment

$L_{\gamma} = 4\pi D_L ^2 \int_{E_{min}}^{E_{max}} E \dfrac{dN}{dE} dE$

$D_L = 134.1$ $Mpc$

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy import units as u
from scipy import interpolate
from scipy.integrate import quad

In [2]:
emin = 0.1 * u.GeV
emax = 300 * u.GeV
D_l = 1588.6 * u.Mpc
D_l = D_l.to(u.cm)
# emin = emin.to(u.erg)
# emax = emax.to(u.erg)

In [3]:
def e_dnde(e, n0, gamma, e0=1):
    return e * n0 * ((e / e0) ** (-gamma))    # Simple power law
    

dirs = ["1-20150918-20160527", "2-20160913-20170615", "3A-20170919-20180123", "3B-20180206-20180622", "4-20180918-20190528", "5-20190928-20200608", "6-20200915-20210503", "7-20211101-20220619", "8-20220818-20230223", "all-20150918-20230223"]


In [4]:
# dir = '3B-20180206-20180622/'
dir = dirs[0] + '/'

# lc_file_path = 'Output/Light_curve_001/4fgl_j0854.8+2006_lightcurve.fits'
lc_file_path = 'Output/Results.fits'
hdul = fits.open('./' + dir+lc_file_path)
lc = hdul[1].data
Table(lc).colnames

for i in range(10):
    print(lc['param_names'][0][i] + '        '+ str(lc['param_values'][0][i]) + '  +/- ' + str(lc['param_errors'][0][i]))

norm        2.2210158035112645e-11  +/- 9.59673342274049e-13
alpha        2.14704652530736  +/- 0.03952274217633343
beta        0.10403632657108997  +/- 0.023398313000777073
Eb        710.9868774414062  +/- nan
        nan  +/- nan
        nan  +/- nan
        nan  +/- nan
        nan  +/- nan
        nan  +/- nan
        nan  +/- nan


In [5]:
lc['pivot_energy']

array([6.36597295e+02, 1.56839138e+03, 9.27128166e+02, 2.18555873e+03,
       2.25861926e+03, 7.09942926e+02, 5.21600859e+02, 3.19147772e+03,
       1.49614797e+02, 1.29391044e+03, 1.00000000e+02, 1.85566476e+03,
       1.94246836e+02, 1.05686842e+03, 2.14863438e+03, 2.81576727e+03,
       1.04442459e+03, 2.46468579e+03, 1.36515723e+03, 1.00000000e+03,
       6.44943444e+03, 2.99999133e+05, 1.00000000e+03, 1.00000000e+02,
       2.99999133e+05, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
       1.00000000e+03, 1.00000000e+03, 1.00000000e+03, 1.00000000e+03,
      

In [6]:
Table(lc)['Source_Name', 'pivot_energy']

Source_Name,pivot_energy
str48,float64
4FGL J0854.8+2006,636.5972949790361
4FGL J0856.8+2056,1568.3913775658416
4FGL J0902.4+2051,927.1281660663791
4FGL J0839.4+1803,2185.558734567372
4FGL J0908.9+2311,2258.6192560379554
4FGL J0910.6+2247,709.9429264189738
4FGL J0836.2+2141,521.6008588305431
4FGL J0912.5+1556,3191.477723459491
4FGL J0831.5+1747,149.61479722563868


In [7]:
params = pd.read_csv('param.csv', index_col=0, sep='&', header=0)

In [8]:
params

,TS,Flux,Flux_err,eFlux,eFlux_err,log_N0,log_N0_err,g,g_err
Seg,,,,,,,,,
1,1586.61,10.79,0.70,4.65,0.27,-9.51,0.03,2.14,0.04
2,891.89,4.07,0.05,3.52,0.09,-9.94,0.05,1.92,0.05
3A,155.29,3.44,0.95,1.53,0.26,-9.89,0.16,2.19,0.19
3B,154.60,3.88,0.67,1.86,0.33,-10.06,0.12,1.97,0.12
4,263.04,4.80,0.78,2.02,0.34,-9.91,0.06,2.21,0.10
5,379.65,4.52,0.72,2.60,0.45,-9.96,0.05,2.10,0.07
6,308.48,3.72,0.67,2.06,0.33,-9.97,0.06,2.03,0.08
7,320.09,2.06,0.25,1.95,0.27,-10.20,0.13,1.84,0.12
8,173.44,4.61,0.18,2.15,0.14,-10.07,0.07,1.96,0.08


In [9]:
luminosity = []
pivot_energy = []
norm = []
norm_err = []
alpha = []
alpha_err = []
beta = []
beta_err = []
eb = []
eb_err = []
for i in range(0, len(dirs)):
    idx = dirs[i].split('-')[0]
    print(idx)
    lc_file_path = 'Output/Results.fits'
    hdul = fits.open('./' + dirs[i]+'/'+lc_file_path)
    lc = hdul[1].data
    e0 = lc['pivot_energy'][0] * u.MeV
    n0 = 10 ** (params['log_N0'][idx]) * (1/(u.cm**2 * u.s * u.MeV))
    gamma = params['g'][idx]
    # print(n0, gamma, e0)
    res = 4*np.pi * (D_l**2) * quad(e_dnde, emin.value, emax.value, args=(n0.to(1/(u.cm**2 * u.s * u.GeV)).value, gamma, e0.to(u.GeV).value), epsabs=1e-18, epsrel=1e-18, limit=100000) * u.GeV/(u.cm**2 * u.s)
    print([res.to(u.erg/u.s).value[0], e0.value])
    # print(res)
    luminosity.append(res.to(u.erg/u.s).value[0])
    pivot_energy.append(e0.value)
    norm.append(lc['param_values'][0][0])
    alpha.append(lc['param_values'][0][1])
    beta.append(lc['param_values'][0][2])
    eb.append(lc['param_values'][0][3])
    norm_err.append(lc['param_errors'][0][0])
    alpha_err.append(lc['param_errors'][0][1])
    beta_err.append(lc['param_errors'][0][2])
    eb_err.append(lc['param_errors'][0][3])


1
[3.779723436280221e+47, 636.5972949790361]
2
[2.76265047508079e+47, 720.5858113367099]
3A
[2.7584460323693513e+47, 846.7727370912797]
3B
[2.3138738179280437e+47, 803.7171733140149]
4
[9.818363580475988e+46, 545.9172076941911]
5
[3.0985347394478485e+47, 921.4474012334452]
6
[2.1790507404572623e+47, 745.6911164846375]
7
[3.961045526402456e+47, 1081.0211517434768]
8
[1.7903109598939206e+47, 705.824591196615]
all
[2.1771595687896484e+47, 715.5639475127598]


In [10]:
dirs[0]

'1-20150918-20160527'

In [11]:
big_table = pd.read_csv('final_table.tex', index_col=0, sep = ' ', header=None)

In [12]:
big_table

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,43,44
0,,,,,,,,,,,,,,,,,,,,,
1,&,1586.61,&,$10.79,\pm,0.70$,&,$4.65,\pm,0.27$,...,&,$,0.104,\pm,0.023,$,&,710.99,$,\\
2,&,891.89,&,$4.07,\pm,0.05$,&,$3.52,\pm,0.09$,...,&,$,0.070,\pm,0.003,$,&,710.99,$,\\
3A,&,155.29,&,$3.44,\pm,0.95$,&,$1.53,\pm,0.26$,...,&,$,0.142,\pm,0.084,$,&,710.99,$,\\
3B,&,154.60,&,$3.88,\pm,0.67$,&,$1.86,\pm,0.33$,...,&,$,0.029,\pm,0.039,$,&,710.99,$,\\
4,&,263.04,&,$4.80,\pm,0.78$,&,$2.02,\pm,0.34$,...,&,$,-0.001,\pm,0.043,$,&,710.99,$,\\
5,&,379.65,&,$4.52,\pm,0.72$,&,$2.60,\pm,0.45$,...,&,$,-0.004,\pm,0.030,$,&,710.99,$,\\
6,&,308.48,&,$3.72,\pm,0.67$,&,$2.06,\pm,0.33$,...,&,$,0.054,\pm,0.047,$,&,710.99,$,\\
7,&,320.09,&,$2.06,\pm,0.25$,&,$1.95,\pm,0.27$,...,&,$,0.115,\pm,0.028,$,&,710.99,$,\\
8,&,173.44,&,$4.61,\pm,0.18$,&,$2.15,\pm,0.14$,...,&,$,-0.074,\pm,0.007,$,&,710.99,$,\\


In [13]:
luminosity

[3.779723436280221e+47,
 2.76265047508079e+47,
 2.7584460323693513e+47,
 2.3138738179280437e+47,
 9.818363580475988e+46,
 3.0985347394478485e+47,
 2.1790507404572623e+47,
 3.961045526402456e+47,
 1.7903109598939206e+47,
 2.1771595687896484e+47]

In [14]:
big_table[19] = [' & '] * len(big_table)
big_table[20] = [np.round(luminosity[i]/1e47, 2) for i in range(0, len(dirs))]
big_table[21] = [' & '] * len(big_table)
big_table[22] = [np.round(pivot_energy[i], 2) for i in range(0, len(dirs))]
big_table[23] = [' & '] * len(big_table)
big_table[24] = [' $ '] * len(big_table)
big_table[25] = [np.round(np.log10(norm[i]), 2) for i in range(0, len(dirs))]
big_table[26] = [' \pm '] * len(big_table)
big_table[27] = [np.round(norm_err[i]/norm[i], 2) for i in range(0, len(dirs))]
big_table[28] = [' $ '] * len(big_table)
big_table[29] = [' & '] * len(big_table)
big_table[30] = [' $ '] * len(big_table)
big_table[31] = [np.round(alpha[i], 3) for i in range(0, len(dirs))]
big_table[32] = [' \pm '] * len(big_table)
big_table[33] = [np.round(alpha_err[i], 3) for i in range(0, len(dirs))]
big_table[34] = [' $ '] * len(big_table)
big_table[35] = [' & '] * len(big_table)
big_table[36] = [' $ '] * len(big_table)
big_table[37] = [np.round(beta[i], 3) for i in range(0, len(dirs))]
big_table[38] = [' \pm '] * len(big_table)
big_table[39] = [np.round(beta_err[i], 3) for i in range(0, len(dirs))]
big_table[40] = [' $ '] * len(big_table)
big_table[41] = [' & '] * len(big_table)
big_table[42] = [np.round(eb[i], 2) for i in range(0, len(dirs))]
big_table[43] = [' $ '] * len(big_table)
big_table[44] = [' \\\\'] * len(big_table)

# final_table.drop(columns=[45, 46, 47], inplace=True)

In [15]:
big_table

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,43,44
0,,,,,,,,,,,,,,,,,,,,,
1,&,1586.61,&,$10.79,\pm,0.70$,&,$4.65,\pm,0.27$,...,&,$,0.104,\pm,0.023,$,&,710.99,$,\\
2,&,891.89,&,$4.07,\pm,0.05$,&,$3.52,\pm,0.09$,...,&,$,0.070,\pm,0.003,$,&,710.99,$,\\
3A,&,155.29,&,$3.44,\pm,0.95$,&,$1.53,\pm,0.26$,...,&,$,0.142,\pm,0.084,$,&,710.99,$,\\
3B,&,154.60,&,$3.88,\pm,0.67$,&,$1.86,\pm,0.33$,...,&,$,0.029,\pm,0.039,$,&,710.99,$,\\
4,&,263.04,&,$4.80,\pm,0.78$,&,$2.02,\pm,0.34$,...,&,$,-0.001,\pm,0.043,$,&,710.99,$,\\
5,&,379.65,&,$4.52,\pm,0.72$,&,$2.60,\pm,0.45$,...,&,$,-0.004,\pm,0.030,$,&,710.99,$,\\
6,&,308.48,&,$3.72,\pm,0.67$,&,$2.06,\pm,0.33$,...,&,$,0.054,\pm,0.047,$,&,710.99,$,\\
7,&,320.09,&,$2.06,\pm,0.25$,&,$1.95,\pm,0.27$,...,&,$,0.115,\pm,0.028,$,&,710.99,$,\\
8,&,173.44,&,$4.61,\pm,0.18$,&,$2.15,\pm,0.14$,...,&,$,-0.074,\pm,0.007,$,&,710.99,$,\\


In [16]:
big_table.to_csv('final_table.tex', sep=' ', header=None)

In [17]:
# for i in range(10):
#     print(dirs[i])
#     print(f"\\\\SED Log Parabola parameters: $\log_{{10}} N_0 = {np.round(np.log10(norm[i]), 2)} \pm {np.round(norm_err[i]/norm[i], 2)}$ ; $\\alpha = {np.round(alpha[i], 3)} \pm {np.round(alpha_err[i], 3)}$ ; $\\beta = {np.round(beta[i], 3)} \pm {np.round(beta_err[i], 3)}$ ; $E_b = {np.round(eb[i], 2)}$ MeV")

In [18]:
big_table[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,
1,&,1586.61,&,$10.79,\pm,0.70$,&,$4.65,\pm,0.27$
2,&,891.89,&,$4.07,\pm,0.05$,&,$3.52,\pm,0.09$
3A,&,155.29,&,$3.44,\pm,0.95$,&,$1.53,\pm,0.26$
3B,&,154.60,&,$3.88,\pm,0.67$,&,$1.86,\pm,0.33$
4,&,263.04,&,$4.80,\pm,0.78$,&,$2.02,\pm,0.34$
5,&,379.65,&,$4.52,\pm,0.72$,&,$2.60,\pm,0.45$
6,&,308.48,&,$3.72,\pm,0.67$,&,$2.06,\pm,0.33$
7,&,320.09,&,$2.06,\pm,0.25$,&,$1.95,\pm,0.27$
8,&,173.44,&,$4.61,\pm,0.18$,&,$2.15,\pm,0.14$


In [21]:
table1 = big_table.copy()[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
table1[11] = [' & '] * len(table1)
table1[12] = [np.round(luminosity[i]/1e47, 2) for i in range(0, len(dirs))]
table1[13] = [' & '] * len(table1)
table1[14] = [np.round(pivot_energy[i], 2) for i in range(0, len(dirs))]
table1[15] = [' \\\\ '] * len(table1)

In [22]:
table1

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,
1,&,1586.61,&,$10.79,\pm,0.70$,&,$4.65,\pm,0.27$,&,3.78,&,636.60,\\
2,&,891.89,&,$4.07,\pm,0.05$,&,$3.52,\pm,0.09$,&,2.76,&,720.59,\\
3A,&,155.29,&,$3.44,\pm,0.95$,&,$1.53,\pm,0.26$,&,2.76,&,846.77,\\
3B,&,154.60,&,$3.88,\pm,0.67$,&,$1.86,\pm,0.33$,&,2.31,&,803.72,\\
4,&,263.04,&,$4.80,\pm,0.78$,&,$2.02,\pm,0.34$,&,0.98,&,545.92,\\
5,&,379.65,&,$4.52,\pm,0.72$,&,$2.60,\pm,0.45$,&,3.10,&,921.45,\\
6,&,308.48,&,$3.72,\pm,0.67$,&,$2.06,\pm,0.33$,&,2.18,&,745.69,\\
7,&,320.09,&,$2.06,\pm,0.25$,&,$1.95,\pm,0.27$,&,3.96,&,1081.02,\\
8,&,173.44,&,$4.61,\pm,0.18$,&,$2.15,\pm,0.14$,&,1.79,&,705.82,\\


In [27]:
table2 = big_table.copy()[np.arange(12, 45)]

In [30]:
table2.drop(columns=[20, 21, 22, 23], inplace=True)

In [31]:
table1.to_csv('table1.tex', sep=' ', header=None)
table2.to_csv('table2.tex', sep=' ', header=None)